# Clasificación de estado de presión de llantas vehículos a partir de imágenes

# 1. Cargando Archivos

In [ ]:
import os
os.getcwd()
import cv2
from google.colab.patches import cv2_imshow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
flat_class = []
for image in os.listdir('flat.class'):
  flat_class.append(cv2.imread(os.path.join('flat.class', image)))

In [ ]:
full_class = []
for image in os.listdir('full.class'):
  full_class.append(cv2.imread(os.path.join('full.class', image)))

In [ ]:
cv2_imshow(flat_class[0]), cv2_imshow(full_class[0]) 

In [ ]:
print(flat_class[0].shape)
print(len(flat_class))

print(full_class[0].shape)
print(len(full_class))

In [ ]:
flat_gray = []
full_gray = []
full_data = []
for img in flat_class:
  flat_gray.append(np.array(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)).flatten())
  full_data.append(np.array(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)).flatten())
for img in full_class:
  full_gray.append(np.array(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)).flatten())
  full_data.append(np.array(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)).flatten())

flat_gray = np.array(flat_gray)
full_gray = np.array(full_gray)
full_data = np.array(full_data)

In [ ]:
print(flat_gray.shape)
print(len(flat_gray))

print(full_gray.shape)
print(len(full_gray))

print(full_data.shape)
print(len(full_data))

In [ ]:
etiquetas = np.zeros((300,))
etiquetas = np.append(etiquetas, np.ones((300,)))
etiquetas.shape